In [1]:
from keras.applications import MobileNet
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# MobileNet is designed to work with images of dim 224,224
img_rows, img_cols = 224, 224
num_classes = 5
batch_size = 32
nb_train_samples = 24256
nb_validation_samples = 3006

epochs = 25

In [3]:
train_data_dir = '../input/emotiondetection/fer2013/train'
validation_data_dir = '../input/emotiondetection/fer2013//validation'

In [4]:
MobileNet = MobileNet(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, 3))

17227776/17225924 [==============================] - 0s 0us/step


In [5]:
# Here we freeze the last 4 layers
# Layers are set to trainable as True by default

for layer in MobileNet.layers:
    layer.trainable = True

In [6]:
# Let's print our layers
for (i, layer) in enumerate(MobileNet.layers):
    print(str(i), layer.__class__.__name__, layer.trainable)

0 InputLayer True
1 ZeroPadding2D True
2 Conv2D True
3 BatchNormalization True
4 ReLU True
5 DepthwiseConv2D True
6 BatchNormalization True
7 ReLU True
8 Conv2D True
9 BatchNormalization True
10 ReLU True
11 ZeroPadding2D True
12 DepthwiseConv2D True
13 BatchNormalization True
14 ReLU True
15 Conv2D True
16 BatchNormalization True
17 ReLU True
18 DepthwiseConv2D True
19 BatchNormalization True
20 ReLU True
21 Conv2D True
22 BatchNormalization True
23 ReLU True
24 ZeroPadding2D True
25 DepthwiseConv2D True
26 BatchNormalization True
27 ReLU True
28 Conv2D True
29 BatchNormalization True
30 ReLU True
31 DepthwiseConv2D True
32 BatchNormalization True
33 ReLU True
34 Conv2D True
35 BatchNormalization True
36 ReLU True
37 ZeroPadding2D True
38 DepthwiseConv2D True
39 BatchNormalization True
40 ReLU True
41 Conv2D True
42 BatchNormalization True
43 ReLU True
44 DepthwiseConv2D True
45 BatchNormalization True
46 ReLU True
47 Conv2D True
48 BatchNormalization True
49 ReLU True
50 DepthwiseCon

In [7]:
def addTopModelMobileNet(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024, activation='relu')(top_model)

    top_model = Dense(1024, activation='relu')(top_model)

    top_model = Dense(512, activation='relu')(top_model)

    top_model = Dense(num_classes, activation='softmax')(top_model)

    return top_model

In [8]:
FC_Head = addTopModelMobileNet(MobileNet, num_classes)

In [9]:
model = Model(inputs=MobileNet.input, outputs=FC_Head)

In [10]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [11]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1. / 255)

In [12]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode='categorical')

Found 24256 images belonging to 5 classes.
Found 3006 images belonging to 5 classes.


In [13]:
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [14]:
checkpoint = ModelCheckpoint(
    'emotion_face_mobilNet.h5',
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose=1)

earlystop = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=10,
    verbose=1, restore_best_weights=True)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=5,
                                            verbose=1,
                                            factor=0.2,
                                            min_lr=0.0001)

callbacks = [earlystop, checkpoint, learning_rate_reduction]

In [15]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy']
              )

In [16]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/25
758/758 [==============================] - 405s 535ms/step - loss: 1.1155 - accuracy: 0.5554 - val_loss: 1.3839 - val_accuracy: 0.5642

Epoch 00001: val_loss improved from inf to 1.38386, saving model to emotion_face_mobilNet.h5


/opt/conda/lib/python3.7/site-packages/keras/callbacks/callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Epoch 2/25
758/758 [==============================] - 389s 513ms/step - loss: 0.9322 - accuracy: 0.6404 - val_loss: 1.8176 - val_accuracy: 0.6056

Epoch 00002: val_loss did not improve from 1.38386
Epoch 3/25
758/758 [==============================] - 360s 475ms/step - loss: 0.8601 - accuracy: 0.6668 - val_loss: 1.3318 - val_accuracy: 0.6022

Epoch 00003: val_loss improved from 1.38386 to 1.33183, saving model to emotion_face_mobilNet.h5
Epoch 4/25
758/758 [==============================] - 355s 468ms/step - loss: 0.8266 - accuracy: 0.6859 - val_loss: 1.5076 - val_accuracy: 0.6036

Epoch 00004: val_loss did not improve from 1.33183
Epoch 5/25
758/758 [==============================] - 358s 473ms/step - loss: 0.7920 - accuracy: 0.7002 - val_loss: 1.3474 - val_accuracy: 0.6113

Epoch 00005: val_loss did not improve from 1.33183
Epoch 6/25
758/758 [==============================] - 365s 481ms/step - loss: 0.7602 - accuracy: 0.7100 - val_loss: 0.6017 - val_accuracy: 0.6308

Epoch 00006: va